In [1]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
import re

In [2]:
def get_tables(path):
    """"
    :param: path of the data base
    :return: data frame with all the data, united by 'uuid'.
    """
    engine = create_engine(f'sqlite:///{path}')
    df_personal_info = pd.read_sql("SELECT * FROM personal_info", engine)
    df_country_info = pd.read_sql("SELECT * FROM country_info", engine)
    df_career_info = pd.read_sql("SELECT * FROM career_info", engine)
    df_poll_info = pd.read_sql("SELECT * FROM poll_info", engine)

    # getting the complete data frame
    dfs_list = [df_personal_info, df_country_info, df_career_info, df_poll_info]
    df_info = reduce(lambda left, right: pd.merge(left, right, on='uuid'), dfs_list)

    # exporting the data frame to processed data folder.
    #df_info.to_csv(f'../../data/processed/info_dataframe.csv', index=True)

    return df_info

In [3]:
get_tables('../data/raw/raw_data_project_m1.db')

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39,SK,urban,high,yes,847165cfda6b1dc82ae22b967da8af2f,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65,SK,urban,high,yes,a4d5b8b38f9513825d0d94a981ebe962,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile,SK,Non-Rural,low,no,None,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65,SK,city,low,yes,775190277a849cba701b306a7b374c0a,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [4]:
path = '../data/raw/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')
df_personal_info = pd.read_sql("SELECT * FROM personal_info", engine)
df_country_info = pd.read_sql("SELECT * FROM country_info", engine)
df_career_info = pd.read_sql("SELECT * FROM career_info", engine)
df_poll_info = pd.read_sql("SELECT * FROM poll_info", engine)

# getting the complete data frame
dfs_list = [df_personal_info, df_country_info, df_career_info, df_poll_info]
df_complete = reduce(lambda left, right: pd.merge(left, right, on='uuid'), dfs_list)

# exporting the data frame to processed data folder.
#df_complete.to_csv(f'./data/processed/complete_dataframe.csv', index=True)
df_complete

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39,SK,urban,high,yes,847165cfda6b1dc82ae22b967da8af2f,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65,SK,urban,high,yes,a4d5b8b38f9513825d0d94a981ebe962,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile,SK,Non-Rural,low,no,None,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65,SK,city,low,yes,775190277a849cba701b306a7b374c0a,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [5]:
# cleaning personal_info gender column.
df_complete['gender'] = df_complete['gender'].astype("string").str.capitalize()
df_complete['gender'] = df_complete['gender'].replace('Fem', 'Female')

In [6]:
# cleaning personal_info age_group column.
df_complete['age_group'] = df_complete['age_group'].replace('juvenile', '14_25')

In [7]:
# cleaning personal_info age column.
df_complete['age'] = df_complete['age'].astype("string")
df_complete['age'] = df_complete['age'].str.replace(' years old', '')

for x in range(1980, 2050):
    df_complete['age'] = df_complete['age'].str.replace(f'{x}', f'{2016 - x}')

df_complete['age'] = df_complete['age'].astype('int64')

In [8]:
df_complete

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Female,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37,Female,nO,26_39,SK,urban,high,yes,847165cfda6b1dc82ae22b967da8af2f,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53,Male,yES,40_65,SK,urban,high,yes,a4d5b8b38f9513825d0d94a981ebe962,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,24,Male,NO,14_25,SK,Non-Rural,low,no,None,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47,Male,yES,40_65,SK,city,low,yes,775190277a849cba701b306a7b374c0a,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [9]:
#Hacemos filtro para ver mejor los elemntos correspondientes a 'juvenile'. Max y min pertenecen al bin 14-25, 
#porque 2016-1991 = 25 y 2016-2002=14, por lo que entendemos que todos pertecen a la msima categoría.
#juvenile_filter = df_personal_info['age_group'].str.contains("juvenile")
#print(df_personal_info[juvenile_filter].max())
#print(df_personal_info[juvenile_filter].min())

In [10]:
# cleaning country_info rural column.
df_complete['rural'] = df_complete['rural'].str.lower()

In [11]:
# cleaning career_info dem_education_level column.
df_complete['dem_education_level'] = df_complete['dem_education_level'].replace('no', 'no education')
df_complete['dem_education_level'] = df_complete['dem_education_level'].fillna('no education')

In [12]:
# cleaning career_info dem_education_level column.
df_complete['dem_education_level'] = df_complete['dem_education_level'].replace('no', 'no education')
df_complete['dem_education_level'] = df_complete['dem_education_level'].fillna('no education')

In [13]:
# cleaning def_has_chidren
df_complete['dem_has_children'] = df_complete['dem_has_children'].str.lower()
df_complete.head()

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,no,40_65,AT,countryside,no education,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,no,26_39,AT,city,no education,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,yes,40_65,AT,country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Female,yes,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [14]:
#connecting API
import json
import requests

In [15]:
response = requests.get(f'http://api.dataatwork.org/v1/jobs') #ESTO NO LO HE PUESTO EN PIPELINE
json_data = response.json()
#problema: solo me salen 20

In [16]:
#response = requests.get('http://api.dataatwork.org/v1/jobs/uuid')
#results = response.json()
#print(len(results))
#solo saca 21 pero necesito más. lo que vamos a hacer es que nos saque los datos 
# para cada uuid, para eso necesitamos saber cuántos uuids hay.

In [17]:
job_code_unique = df_complete['normalized_job_code'].unique()
#len(job_code_unique)

In [18]:
list_uuid_key = []
list_title_value = []

for code in job_code_unique:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{code}').json()
    if list(response.keys())[0] == 'error':
        pass
    else:
        list_uuid_key.append(response['uuid'])
        list_title_value.append(response['title'])

In [19]:
dict_uuid_jobs_title = dict(zip(list_uuid_key , list_title_value))

In [20]:
df_complete['Job Title'] = df_complete['normalized_job_code']

for uuid, title in dict_uuid_jobs_title.items():
    df_complete.loc[df_complete['normalized_job_code'] == uuid , 'Job Title'] = title

In [21]:
df_complete['Job Title'] = df_complete['Job Title'].fillna('No job')

In [22]:
df_complete.head(3)

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Job Title
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,no,40_65,AT,countryside,no education,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above,No job
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,no,26_39,AT,city,no education,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,No job


Web scraping

In [23]:
import requests
from bs4 import BeautifulSoup

In [24]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')

In [25]:
table = soup.find_all('div' , {'class': 'mw-content-ltr'})[0]
table_countries = table.find_all('table')
all_info_countries = [info.text for info in table_countries]

In [26]:
# cleaning the info countries.
countries_without_spaces = [re.sub(r'\s' , '' , x) for x in all_info_countries]
countries_without_symbols = [re.sub(r'\*' , '' , x) for x in countries_without_spaces]
countries_without_squarebrackets = [re.sub(r'\[\d\]' , '' , x) for x in countries_without_symbols]
countries_clean = ''.join(countries_without_squarebrackets) #unimos todo en una string para poder hacer split

In [27]:
country_value = re.split(r'\(\w{0,7}\)' , countries_clean)
country_key = re.findall(r'\(\w{0,7}\)' , countries_clean) #extract the codes
country_key = [re.sub(r'\(|\)' , '' , x) for x in country_key] #without parenthesis.
dic_countries = dict(zip(country_key, country_value))

In [28]:
df_complete['Country'] = df_complete['country_code']

for code, name in dic_countries.items():
    df_complete.loc[df_complete['country_code'] == code , 'Country'] = name

In [29]:
df_complete.head()

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Job Title,Country
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,no,40_65,AT,countryside,no education,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above,No job,Austria
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,Austria
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,no,26_39,AT,city,no education,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,No job,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,yes,40_65,AT,country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Female,yes,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,Austria


In [30]:
# renaming columns
df_complete.rename(columns={'age': 'Age',
                           'gender': 'Gender',
                           'dem_has_children': 'Childrens',
                           'age_group': 'Age Group',
                           'country_code': 'Country Code',
                           'rural': 'Area',
                           'dem_education_level': 'Education Level',
                           'dem_full_time_job': 'Full Time Job',
                           'normalized_job_code': 'Job Code',
                           'question_bbi_2016wave4_basicincome_awareness': 'Q1: awareness',
                           'question_bbi_2016wave4_basicincome_vote': 'Q2: vote',
                           'question_bbi_2016wave4_basicincome_effect': 'Q3: effect',
                           'question_bbi_2016wave4_basicincome_argumentsfor': 'Q4: aurguments for',
                           'question_bbi_2016wave4_basicincome_argumentsagainst': 'Q5: arguments againts'}, inplace=True)

In [56]:
# adding quantity column
df_complete['Quantity'] = df_complete.groupby('uuid')['uuid'].transform('count').astype("int64")

In [57]:
df_complete.head()

,uuid,Age,Gender,Childrens,Age Group,Country Code,Area,Education Level,Full Time Job,Job Code,Q1: awareness,Q2: vote,Q3: effect,Q4: aurguments for,Q5: arguments againts,Job Title,Country,Quantity,Total Votes Per Country,Percentage
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,no,40_65,AT,countryside,no education,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above,No job,Austria,1,133,0.0015%
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,Austria,1,133,0.0015%
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,no,26_39,AT,city,no education,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,No job,Austria,1,133,0.0015%
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,yes,40_65,AT,country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,Austria,1,133,0.0015%
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Female,yes,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,Austria,1,133,0.0015%


In [94]:

df_complete[['Country', 'Job Title', 'Age', 'Quantity', 'Percentage']].groupby(['Country', 'Job Title']).agg(Age = ('Age', 'mean'), Quantity = ('Quantity', 'sum'), Percentage=('Percentage', 'sum'))
# ahora el problema e spercenage

Age  \
Country Job Title                                                       
Austria Automatic Data Processing Customer Liaison (ADP...  35.500000   
        Automatic Data Processing Planner                   52.500000   
        Automatic Data Processing Systems Security Spec...  37.000000   
        Clinical Data Specialist                            39.000000   
        Computer or Data Processing Systems Consultant      44.666667   
...                                                               ...   
Sweden  Naval Tactical Data System--Combat Information ...  27.000000   
        No job                                              31.074074   
        Oracle Database Administrator (Oracle DBA)          35.000000   
        SQL Database Administrator                          45.000000   
        Survey Data Technician                              46.000000   

                                                            Quantity  \
Country Job Title                                                      
Austria Automatic Data Processing Customer Liaison (ADP...         2   
        Automatic Data Processing Planner                          2   
        Automatic Data Processing Systems Security Spec...         2   
        Clinical Data Specialist                                   1   
        Computer or Data Processing Systems Consultant             3   
...                                                              ...   
Sweden  Naval Tactical Data System--Combat Information ...         1   
        No job                                                    81   
        Oracle Database Administrator (Oracle DBA)                 1   
        SQL Database Administrator                                 1   
        Survey Data Technician                                     1   

                                                                                                   Percentage  
Country Job Title                                                                                              
Austria Automatic Data Processing Customer Liaison (ADP...                                     0.0015%0.0015%  
        Automatic Data Processing Planner                                                      0.0015%0.0015%  
        Automatic Data Processing Systems Security Spec...                                     0.0015%0.0015%  
        Clinical Data Specialist                                                                      0.0015%  
        Computer or Data Processing Systems Consultant                                  0.0015%0.0015%0.0015%  
...                                                                                                       ...  
Sweden  Naval Tactical Data System--Combat Information ...                                            0.0020%  
        No job                                              0.0020%0.0020%0.0020%0.0020%0.0020%0.0020%0.00...  
        Oracle Database Administrator (Oracle DBA)                                                    0.0020%  
        SQL Database Administrator                                                                    0.0020%  
        Survey Data Technician                                                                        0.0020%  

[1866 rows x 3 columns]

In [95]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9649 entries, 0 to 9648
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uuid                     9649 non-null   object
 1   Age                      9649 non-null   int64 
 2   Gender                   9649 non-null   string
 3   Childrens                9649 non-null   object
 4   Age Group                9649 non-null   object
 5   Country Code             9649 non-null   object
 6   Area                     9649 non-null   object
 7   Education Level          9649 non-null   object
 8   Full Time Job            9649 non-null   object
 9   Job Code                 5702 non-null   object
 10  Q1: awareness            9649 non-null   object
 11  Q2: vote                 9649 non-null   object
 12  Q3: effect               9649 non-null   object
 13  Q4: aurguments for       9649 non-null   object
 14  Q5: arguments againts    9649 non-null  

In [37]:
# adding percentage column
# votos por país

list_countries =df_complete['Country'].unique().tolist()
list_countries

['Austria',
 'Belgium',
 'Bulgaria',
 'Cyprus',
 'Czechia',
 'Germany',
 'Denmark',
 'Estonia',
 'Spain',
 'Finland',
 'France',
 'GB',
 'GR',
 'Croatia',
 'Hungary',
 'Ireland',
 'Italy',
 'Lithuania',
 'Luxembourg',
 'Latvia',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Sweden',
 'Slovenia',
 'Slovakia']

In [38]:
# total people per country column.
df_complete['Total Votes Per Country'] = 0

for country in list_countries:
    df_complete.loc[df_complete['Country'] == country , 'Total Votes Per Country'] = df_complete.loc[df_complete['Country'].str.contains(country) , 'Quantity'].sum()

In [101]:
df_complete['Percentage'] = (df_complete['Total Votes Per Country'] / df_complete['Total Votes Per Country'].sum()) 
#df_complete['Percentage'] = df_complete['Percentage'].astype(float).map(lambda x: '{:.4%}'.format(x)) # adding the %
df_complete.head()

,uuid,Age,Gender,Childrens,Age Group,Country Code,Area,Education Level,Full Time Job,Job Code,Q1: awareness,Q2: vote,Q3: effect,Q4: aurguments for,Q5: arguments againts,Job Title,Country,Quantity,Total Votes Per Country,Percentage
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,Male,no,40_65,AT,countryside,no education,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above,No job,Austria,1,133,0.000015
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,Austria,1,133,0.000015
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,Male,no,26_39,AT,city,no education,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,No job,Austria,1,133,0.000015
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,yes,40_65,AT,country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,Austria,1,133,0.000015
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Female,yes,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,Austria,1,133,0.000015


In [126]:
df_result = df_complete[['Country', 'Job Title', 'Age', 'Quantity', 'Percentage']].groupby(['Country', 'Job Title'], as_index=False).agg(Age = ('Age', 'mean'), Quantity = ('Quantity', 'sum'), Percentage=('Percentage', 'sum'))
#df_result['Age'] = df_result['Age'].astype(float).round(3)
df_result = df_result.round({'Age': 2, 'Percentage': 6})
df_result

,Country,Job Title,Age,Quantity,Percentage
0,Austria,Automatic Data Processing Customer Liaison (AD...,35.50,2,0.000031
1,Austria,Automatic Data Processing Planner,52.50,2,0.000031
2,Austria,Automatic Data Processing Systems Security Spe...,37.00,2,0.000031
3,Austria,Clinical Data Specialist,39.00,1,0.000015
4,Austria,Computer or Data Processing Systems Consultant,44.67,3,0.000046
...,...,...,...,...,...
1861,Sweden,Naval Tactical Data System--Combat Information...,27.00,1,0.000020
1862,Sweden,No job,31.07,81,0.001648
1863,Sweden,Oracle Database Administrator (Oracle DBA),35.00,1,0.000020
1864,Sweden,SQL Database Administrator,45.00,1,0.000020


In [123]:
df_result[df_result['Country'] == 'Spain']

,Country,Job Title,Age,Quantity,Percentage
1669,Spain,Analytical Data Miner,22.000000,1,0.000117
1670,Spain,Automatic Data Processing Customer Liaison (AD...,39.727273,11,0.001285
1671,Spain,Automatic Data Processing Planner,40.000000,9,0.001051
1672,Spain,Automatic Data Processing Planner (ADP Planner),46.000000,11,0.001285
1673,Spain,Automatic Data Processing Systems Security Spe...,46.428571,7,0.000818
...,...,...,...,...,...
1793,Spain,SQL Database Administrator,49.375000,8,0.000935
1794,Spain,Scientific Database Curator,44.000000,2,0.000234
1795,Spain,Survey Data Technician,19.000000,1,0.000117
1796,Spain,Voice and Data Technician,32.500000,2,0.000234


In [116]:
list_countries = df_complete['Country'].unique().tolist()
country = 'Spain'

if country in list_countries:
    df_result_per_country = df_result[df_result['Country'] == f'{country}']
    print('exporting result_per_country.csv file')

KeyError: 'Country'

In [117]:
df_result_per_country

NameError: name 'df_result_per_country' is not defined

In [114]:
#df_result['Percentage'] = df_result['Percentage'].astype(float).map(lambda x: '{:.4%}'.format(x)) # adding the %


In [ ]:
# choosing and printing (should export).

country_choosed = input(print(f'Would you like to see the data for: \n- all countries \n- for: {list_countries}?'))

if country_choosed == 'all countries':
    print('exporting all_countries_data.csv file...')
    df_result.to_csv('../data/result/all_countries_data.csv')
    print('file exported!')
if country_choosed in list_countries:
    df_result_per_country = df_result[df_result['Country'] == (f'{country_choosed}')]
    print('exporting result_per_country.csv file')
    df_result_per_country.to_csv('../data/result/result_per_country.csv')
    print('file exported!')

In [ ]:
#bonus 1: table about votes.

In [ ]:
# cleaniin Q3 column
df_complete['Q3: effect'] = df_complete['Q3: effect'].astype("string")
df_complete['Q3: effect'] = df_complete['Q3: effect'].replace("‰Û_ ", "")

In [ ]:
df_complete['Q3: effect'] = [re.sub(r'\‰Û_', '', x) for x in df_complete['Q3: effect']]

In [ ]:
df_complete.head(2)

In [ ]:
# Q4 vamos a unirlo todo en una sola lista para ver los valores únicos.
arguments_for = [info for info in df_complete['Q4: aurguments for']]

In [ ]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [ ]:
all_arguments = convert_list_to_string(arguments_for)

In [ ]:
arguments_split_mayus = re.findall('[A-Z][^A-Z]*', all_arguments)

In [ ]:
arguments_without_verticalbar = [re.sub('\ \| +', '', x) for x in arguments_split_mayus]

In [ ]:
arguments_cleaned = [re.sub(' [^ ]*$', '', x) for x in arguments_without_verticalbar]
arguments_cleaned_set = set(arguments_cleaned)

In [ ]:
arguments_cleaned_set

In [ ]:
number_of_pro_arguments(arguments_cleaned)

In [ ]:
more_equality = sum(map(lambda x : x=='It creates more equality of' or x=='It creates more equality of opportunity', arguments_cleaned))
financial_independence = sum(map(lambda x : x == 'It encourages financial independence and' or x == 'It encourages financial independence and self-responsibility', arguments_cleaned))
household_appreciation = sum(map(lambda x : x== 'It increases appreciation for household work and' or x == 'It increases appreciation for household work and volunteering', arguments_cleaned))
solidarity = sum(map(lambda x : x== 'It increases solidarity, because it is funded by' or x == 'It increases solidarity, because it is funded by everyone', arguments_cleaned))
reduces_anxiety = sum(map(lambda x : x== 'It reduces anxiety about financing basic' or x == 'It reduces anxiety about financing basic needs', arguments_cleaned))
reduces_boreaucracy_expenses = sum(map(lambda x : x== 'It reduces bureaucracy and administrative' or x == 'It reduces bureaucracy and administrative expenses', arguments_cleaned))
none_of_above = sum(map(lambda x: x == 'None of the above', arguments_cleaned))

In [ ]:
#number_of_votes = pd.DataFrame(votes, columns = ['Number of pro arguments', 'Number of cons arguments']
#votes = [['In favor', number_pro_arguments], ['Against': Y]]

In [ ]:
number_pro_favor_arguments = more_equality + financial_independence + household_appreciation + solidarity + reduces_anxiety + reduces_boreaucracy_expenses
number_pro_against_arguments = none_of_above

In [ ]:
number_of_votes

In [ ]:
#Q5: hacemos todo el proceso de nuevo

In [ ]:
arguments_against = [info for info in df_complete['Q5: arguments againts']]
arguments_against_together = ' '.join(arguments_against)
arguments_against_split_mayus = re.findall('[A-Z][^A-Z]*', arguments_against_together)
arguments_against_without_verticalbar = [re.sub('\ \| +', '', x) for x in arguments_against_split_mayus]
arguments_against_cleaned = [re.sub(' [^ ]*$', '', x) for x in arguments_against_without_verticalbar]
arguments_against_cleaned_set = set(arguments_against_cleaned)

In [ ]:
arguments_against_cleaned_set

In [ ]:
foreigners = sum(map(lambda x : x=='Foreigners might come to my country and take advantage of the' or x=='Foreigners might come to my country and take advantage of the benefit', arguments_against_cleaned))
state_dependence = sum(map(lambda x : x == 'It increases dependence on the' or x == 'It increases dependence on the state', arguments_against_cleaned))
against_merit = sum(map(lambda x : x== 'It is against the principle of linking merit and' or x == 'It is against the principle of linking merit and reward', arguments_against_cleaned))
impossible_finance = sum(map(lambda x : x== 'It is impossible to' or x == 'It is impossible to finance', arguments_against_cleaned))
stop_working = sum(map(lambda x : x== 'It might encourage people to stop' or x == 'It might encourage people to stop working', arguments_against_cleaned))
only_needed = sum(map(lambda x : x== 'Only the people who need it most should get something from the' or x == 'Only the people who need it most should get something from the state', arguments_against_cleaned))
none_of_above_ag = sum(map(lambda x: x == 'None of the above', arguments_against_cleaned))

In [ ]:
number_cons_infavor = foreigners + state_dependence + against_merit + impossible_finance + stop_working + only_needed
number_cons_against = none_of_above_ag

In [ ]:
votes = [['In favor', number_pro_arguments, number_cons_infavor], ['Against', none_of_above, number_cons_against]]
number_of_votes = pd.DataFrame(votes, columns = ['Position', 'Number of pro arguments', 'Number of cons arguments'])

In [ ]:
number_of_votes